<a href="https://colab.research.google.com/github/harley1983/Final-assignment-ISYS2001/blob/main/WeatherWise_Advisor_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# WeatherWise Advisor - Google Colab Notebook

This notebook presents the WeatherWise Advisor application, a Python-based weather tool that fetches real-time weather data, processes natural language queries, and generates visualizations. It is organized into modular sections as per the assignment requirements for the Business Programming unit.

Developed with AI assistance (e.g., ChatGPT, Claude) for code generation, debugging, and documentation.

In [ ]:
# Table of Contents
- [Setup and Configuration](#setup-and-configuration)
- [Weather Data Functions](#weather-data-functions)
- [Visualisation Functions](#visualisation-functions)
- [Natural Language Processing](#natural-language-processing)
- [User Interface](#user-interface)
- [Main Application Logic](#main-application-logic)
- [Testing and Examples](#testing-and-examples)

In [ ]:
# Setup and Configuration
This section includes library imports, API key configuration, and initial setup for the WeatherWise Advisor app. All required dependencies are listed, and NLTK data is downloaded if not already present.

In [ ]:
# Import required libraries
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pyinputplus as pyip
import re
from datetime import datetime, timedelta
import json
import os
import platform
import time
import nltk
from nltk.tokenize import word_tokenize

# Download NLTK data if not already present
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

# API configuration (use environment variable for security in production)
API_KEY = "a2d2296beda67fc9855b4eb3559e24b3"  # Replace with your actual API key or use os.getenv()
BASE_URL = "https://api.openweathermap.org/data/2.5"


In [ ]:
    # Download NLTK data if not already present
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt', quiet=True)


In [ ]:
# Weather Data Functions
Functions for retrieving and processing weather data from the OpenWeatherMap API.
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    # Ensure forecast days is within acceptable range
    forecast_days = max(1, min(forecast_days, 5))

    try:
        # Get current weather
        current_url = f"{BASE_URL}/weather?q={location}&units=metric&appid={API_KEY}"
        current_response = requests.get(current_url)

        if current_response.status_code != 200:
            if current_response.status_code == 404:
                return {"error": "Location not found. Please check the spelling and try again."}
            else:
                return {"error": f"API Error: {current_response.status_code}"}

        current_data = current_response.json()

        # Get forecast data
        forecast_url = f"{BASE_URL}/forecast?q={location}&units=metric&appid={API_KEY}"
        forecast_response = requests.get(forecast_url)

        if forecast_response.status_code != 200:
            return {"error": f"Forecast API Error: {forecast_response.status_code}"}

        forecast_data = forecast_response.json()

        # Process and structure the data
        processed_data = {
            "location": {
                "name": current_data["name"],
                "country": current_data["sys"]["country"],
                "coordinates": {
                    "lat": current_data["coord"]["lat"],
                    "lon": current_data["coord"]["lon"]
                }
            },
            "current": {
                "timestamp": datetime.fromtimestamp(current_data["dt"]).strftime('%Y-%m-%d %H:%M:%S'),
                "temperature": current_data["main"]["temp"],
                "feels_like": current_data["main"]["feels_like"],
                "humidity": current_data["main"]["humidity"],
                "pressure": current_data["main"]["pressure"],
                "wind": {
                    "speed": current_data["wind"]["speed"],
                    "direction": current_data["wind"].get("deg", 0)
                },
                "description": current_data["weather"][0]["description"],
                "main": current_data["weather"][0]["main"],
                "icon": current_data["weather"][0]["icon"],
                "clouds": current_data.get("clouds", {}).get("all", 0),
                "rain": current_data.get("rain", {}).get("1h", 0),
                "visibility": current_data.get("visibility", 0) / 1000  # Convert to km
            },
            "forecast": []
        }

        # Group forecast by day
        daily_forecasts = {}
        today = datetime.now().date()

        for item in forecast_data["list"]:
            dt = datetime.fromtimestamp(item["dt"])
            day = dt.date()

            if (day - today).days >= forecast_days:
                continue

            if day not in daily_forecasts:
                daily_forecasts[day] = []

            daily_forecasts[day].append({
                "timestamp": dt.strftime('%Y-%m-%d %H:%M:%S'),
                "temperature": item["main"]["temp"],
                "feels_like": item["main"]["feels_like"],
                "humidity": item["main"]["humidity"],
                "pressure": item["main"]["pressure"],
                "description": item["weather"][0]["description"],
                "main": item["weather"][0]["main"],
                "icon": item["weather"][0]["icon"],
                "clouds": item["clouds"]["all"],
                "wind": {
                    "speed": item["wind"]["speed"],
                    "direction": item["wind"].get("deg", 0)
                },
                "pop": item.get("pop", 0) * 100,  # Probability of precipitation as percentage
                "rain": item.get("rain", {}).get("3h", 0),
                "snow": item.get("snow", {}).get("3h", 0),
                "hour": dt.hour
            })

        # Calculate daily stats and add to processed data
        for day, forecasts in sorted(daily_forecasts.items()):
            daily_temps = [f["temperature"] for f in forecasts]
            daily_humidity = [f["humidity"] for f in forecasts]
            daily_clouds = [f["clouds"] for f in forecasts]
            daily_pop = [f["pop"] for f in forecasts]

            # Check if we have rain data
            daily_rain = [f["rain"] for f in forecasts if "rain" in f]

            daily_summary = {
                "date": day.strftime('%Y-%m-%d'),
                "day_name": day.strftime('%A'),
                "temperature": {
                    "min": min(daily_temps),
                    "max": max(daily_temps),
                    "avg": sum(daily_temps) / len(daily_temps)
                },
                "humidity": {
                    "min": min(daily_humidity),
                    "max": max(daily_humidity),
                    "avg": sum(daily_humidity) / len(daily_humidity)
                },
                "clouds": {
                    "min": min(daily_clouds),
                    "max": max(daily_clouds),
                    "avg": sum(daily_clouds) / len(daily_clouds)
                },
                "precipitation_chance": max(daily_pop),
                "hourly": forecasts
            }

            if daily_rain:
                daily_summary["rain"] = {
                    "total": sum(daily_rain),
                    "max": max(daily_rain)
                }

            processed_data["forecast"].append(daily_summary)

        return processed_data
    except requests.RequestException as e:
        return {"error": f"Network error: {str(e)}"}
    except Exception as e:
        return {"error": f"Error processing weather data: {str(e)}"}


In [ ]:
# Visualisation Functions
This section includes functions for creating visual representations of weather data, such as temperature, precipitation, and wind charts, using matplotlib and seaborn for enhanced styling.
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    if "error" in weather_data:
        print(f"Error: {weather_data['error']}")
        return None

    # Set up the figure and styling
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 6))

    location_name = f"{weather_data['location']['name']}, {weather_data['location']['country']}"

    # Current temperature reference
    current_temp = weather_data['current']['temperature']

    # Extract data for plotting
    dates = []
    temp_max = []
    temp_min = []
    temp_avg = []

    for day in weather_data['forecast']:
        dates.append(day['day_name'])
        temp_min.append(day['temperature']['min'])
        temp_max.append(day['temperature']['max'])
        temp_avg.append(day['temperature']['avg'])

    # Create x position for bars
    x = np.arange(len(dates))
    width = 0.25

    # Plot bars
    ax.bar(x - width, temp_min, width, label='Min Temp (°C)', color='lightblue', alpha=0.7)
    ax.bar(x, temp_avg, width, label='Avg Temp (°C)', color='skyblue', alpha=0.7)
    ax.bar(x + width, temp_max, width, label='Max Temp (°C)', color='darkblue', alpha=0.7)

    # Add a horizontal line for current temperature
    ax.axhline(y=current_temp, linestyle='--', color='red', alpha=0.7)
    ax.text(len(dates)-1, current_temp, f'Current: {current_temp:.1f}°C',
            va='bottom', ha='right', color='red', fontweight='bold')

    # Set labels and title
    ax.set_xlabel('Day', fontsize=12)
    ax.set_ylabel('Temperature (°C)', fontsize=12)
    ax.set_title(f'Temperature Forecast for {location_name}', fontsize=14, fontweight='bold')

    # Set x-ticks
    ax.set_xticks(x)
    ax.set_xticklabels(dates)

    # Add legend
    ax.legend()

    # Add grid for better readability
    ax.grid(True, axis='y', linestyle='--', alpha=0.7)

    # Add overall range annotation
    overall_min = min(temp_min)
    overall_max = max(temp_max)
    ax.annotate(f'Temperature range: {overall_min:.1f}°C - {overall_max:.1f}°C',
                xy=(0.02, 0.95), xycoords='axes fraction',
                fontsize=11, backgroundcolor='white', alpha=0.8)

    plt.tight_layout()

    if output_type == 'figure':
        return fig
    else:
        plt.show()
        plt.close()


In [ ]:
def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    if "error" in weather_data:
        print(f"Error: {weather_data['error']}")
        return None

    # Set up the figure and styling
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax1 = plt.subplots(figsize=(12, 6))

    location_name = f"{weather_data['location']['name']}, {weather_data['location']['country']}"

    # Extract data for plotting
    dates = []
    precip_chance = []
    humidity_avg = []
    rain_amount = []

    for day in weather_data['forecast']:
        dates.append(day['day_name'])
        precip_chance.append(day['precipitation_chance'])
        humidity_avg.append(day['humidity']['avg'])

        # Some locations might not have rain data
        if 'rain' in day:
            rain_amount.append(day['rain']['total'])
        else:
            rain_amount.append(0)

    # Plot precipitation chance as bars
    x = np.arange(len(dates))
    bars = ax1.bar(x, precip_chance, color='skyblue', alpha=0.7, label='Precipitation Chance (%)')

    # Add percentage labels on top of bars
    for i, v in enumerate(precip_chance):
        ax1.text(i, v + 2, f"{v:.0f}%", ha='center', fontsize=9)

    # Set primary axis labels
    ax1.set_xlabel('Day', fontsize=12)
    ax1.set_ylabel('Precipitation Chance (%)', fontsize=12)
    ax1.set_ylim(0, max(100, max(precip_chance) * 1.2))  # Cap at 100% with some headroom

    # Create secondary y-axis for rain amount
    ax2 = ax1.twinx()
    line = ax2.plot(x, rain_amount, 'o-', color='blue', linewidth=2, label='Rainfall (mm)')

    # Add rain amount annotations
    for i, v in enumerate(r

In [ ]:
# User Interface
This section includes functions for displaying weather information in a formatted text output and handling user interactions. These functions are designed for a terminal interface but are adapted here for notebook demonstration.

In [ ]:
def display_current_weather(weather_data):
    """
    Display current weather information in a formatted text output.

    Args:
        weather_data (dict): The processed weather data
    """
    if "error" in weather_data:
        print(f"Error: {weather_data['error']}")
        return

    location = f"{weather_data['location']['name']}, {weather_data['location']['country']}"
    current = weather_data['current']

    print(f"\n{'='*50}")
    print(f"  CURRENT WEATHER FOR {location.upper()}")
    print(f"  {datetime.now().strftime('%A, %B %d, %Y at %H:%M')}")
    print(f"{'='*50}")
    print(f"Temperature: {current['temperature']:.1f}°C (Feels like: {current['feels_like']:.1f}°C)")
    print(f"Conditions: {current['description'].capitalize()}")
    print(f"Humidity: {current['humidity']}%")
    print(f"Pressure: {current['pressure']} hPa")
    print(f"Wind: {current['wind']['speed']} m/s")
    print(f"Visibility: {current['visibility']} km")
    print(f"Cloud Cover: {current['clouds']}%")

    # Add rain information if available
    if current.get('rain', 0) > 0:
        print(f"Rain in Last Hour: {current['rain']} mm")

    print(f"{'='*50}")


In [ ]:
def display_forecast(weather_data):
    """
    Display weather forecast information in a formatted text output.

    Args:
        weather_data (dict): The processed weather data
    """
    if "error" in weather_data:
        print(f"Error: {weather_data['error']}")
        return

    if not weather_data.get('forecast'):
        print("No forecast data available.")
        return

    location = f"{weather_data['location']['name']}, {weather_data['location']['country']}"

    print(f"\n{'='*60}")
    print(f"  WEATHER FORECAST FOR {location.upper()}")
    print(f"{'='*60}")

    for day in weather_data['forecast']:
        print(f"\n{day['day_name']} ({day['date']}):")
        print(f"  Temperature: {day['temperature']['min']:.1f}°C to {day['temperature']['max']:.1f}°C")
        print(f"  Humidity: {day['humidity']['avg']:.0f}% (Range: {day['humidity']['min']}% - {day['humidity']['max']}%)")
        print(f"  Cloud Cover: {day['clouds']['avg']:.0f}%")
        print(f"  Precipitation Chance: {day['precipitation_chance']:.0f}%")

        if 'rain' in day and day['rain']['total'] > 0:
            print(f"  Expected Rainfall: {day['rain']['total']:.1f} mm")

        # Display some hourly details
        print("\n  Hourly forecast highlights:")
        morning = None
        noon = None
        evening = None

        # Find entries close to morning (8-9am), noon (12-1pm), and evening (6-7pm)
        for hour_data in day['hourly']:
            hour = datetime.strptime(hour_data['timestamp'], '%Y-%m-%d %H:%M:%S').hour
            if 8 <= hour <= 9 and not morning:
                morning = hour_data
            elif 12 <= hour <= 13 and not noon:
                noon = hour_data
            elif 18 <= hour <= 19 and not evening:
                evening = hour_data

        time_slots = []
        if morning:
            time_slots.append(("Morning", morning))
        if noon:
            time_slots.append(("Noon", noon))
        if evening:
            time_slots.append(("Evening", evening))

        for time_name, data in time_slots:
            hour = datetime.strptime(data['timestamp'], '%Y-%m-%d %H:%M:%S').hour
            print(f"    {time_name} ({hour}:00): {data['temperature']:.1f}°C, {data['description'].capitalize()}, "
                  f"Wind: {data['wind']['speed']} m/s, Precip: {data['pop']:.0f}%")

    print(f"\n{'='*60}")


In [ ]:
def clear_console():
    """
    Clear the console screen using os module.
    Works on Windows, macOS, and Linux.
    Note: This function is designed for terminal use and may not have an effect in Google Colab.
    """
    import os
    # For Windows
    if os.name == 'nt':
        os.system('cls')
    # For macOS and Linux
    else:
        os.system('clear')


In [ ]:
# Main Application Logic
This section contains the core functionality to run the WeatherWise Advisor app. The `run_weather_advisor()` function provides an interactive menu system for users to access weather data, visualizations, and natural language queries. Adapted for notebook use, it may require manual input or can be run in parts.

In [ ]:
def run_weather_advisor():
    """
    Main function to run the Weather Advisor application using standard input instead of PyInputPlus.
    Note: This is designed for terminal use and may require adaptation for Google Colab. For demonstration, parts can be run manually in separate cells.
    """
    clear_console()

    print("\n" + "="*60)
    print("       WELCOME TO THE WEATHER ADVISOR APPLICATION")
    print("="*60)
    print("\nThis application allows you to:")
    print("  1. Check current weather for any location")
    print("  2. View weather forecasts for up to 5 days")
    print("  3. Ask natural language questions about the weather")
    print("  4. View weather data visualizations")
    print("\nAll data is provided in metric units (°C, m/s, mm, etc.)")
    print("\nLet's get started!")

    # Initialize with a default location
    location = input("\nEnter a location (city name): ")
    weather_data = get_weather_data(location)

    if "error" in weather_data:
        print(f"\nError retrieving weather data: {weather_data['error']}")
        location = input("\nPlease try a different location: ")
        weather_data = get_weather_data(location)
        if "error" in weather_data:
            print(f"\nError retrieving weather data: {weather_data['error']}")
            print("\nExiting application. Please try again later.")
            return

    while True:
        clear_console()

        print("\n" + "="*60)
        print(f"       WEATHER ADVISOR: {weather_data['location']['name'].upper()}, {weather_data['location']['country']}")
        print("="*60)

        menu_options = [
            'View Current Weather',
            'View Forecast',
            'Ask a Weather Question',
            'View Temperature Visualization',
            'View Precipitation Visualization',
            'View Wind Visualization',
            'Change Location',
            'Exit'
        ]

        print("\nWhat would you like to do?")
        for i, option in enumerate(menu_options, 1):
            print(f"{i}. {option}")

        # Input validation for menu choice
        while True:
            try:
                choice = int(input("\nEnter your choice (1-8): "))
                if 1 <= choice <= 8:
                    break
                else:
                    print("Please enter a number between 1 and 8.")
            except ValueError:
                print("Please enter a valid number.")

        if choice == 1:  # View Current Weather
            clear_console()
            display_current_weather(weather_data)
            input("Press Enter to continue...")

        elif choice == 2:  # View Forecast
            clear_console()
            display_forecast(weather_data)
            input("Press Enter to continue...")

        elif choice == 3:  # Ask a Weather Question
            clear_console()
            print("\n" + "="*60)
            print("              ASK A WEATHER QUESTION")
            print("="*60)
            print("\nExamples:")
            print("- Will it rain tomorrow in this city?")
            print("- What's the temperature going to be like this weekend?")
            print("- How windy is it right now?")
            print("- What's the forecast for next Tuesday?")

            question = input("\nYour weather question: ")
            parsed_question = parse_weather_question(question)

            # If no location was found in the question, use the current one
            if not parsed_question["location"]:
                parsed_question["location"] = weather_data["location"]["name"]

            # If the location in the question is different from the current one, fetch new data
            if (parsed_question["location"].lower() != weather_data["location"]["name"].lower() and
                parsed_question["location"].lower() != f"{weather_data['location']['name']}, {weather_data['location']['country']}".lower()):
                print(f"\nFetching weather data for {parsed_question['location']}...")
                new_weather_data = get_weather_data(parsed_question["location"])
                if "error" not in new_weather_data:
                    weather_data = new_weather_data
                else:
                    print(f"\nCouldn't find weather data for {parsed_question['location']}. Using current location instead.")

            response = generate_weather_response(parsed_question, weather_data)
            print(f"\nResponse: {response}")

            input("\nPress Enter to continue...")

        elif choice == 4:  # View Temperature Visualization
            clear_console()
            print("\nGenerating temperature visualization...")
            create_temperature_visualisation(weather_data)
            input("\nPress Enter to continue...")

        elif choice == 5:  # View Precipitation Visualization
            clear_console()
            print("\nGenerating precipitation visualization...")
            create_precipitation_visualisation(weather_data)
            input("\nPress Enter to continue...")

        elif choice == 6:  # View Wind Visualization
            clear_console()
            print("\nGenerating wind visualization...")
            create_wind_visualisation(weather_data)
            input("\nPress Enter to continue...")

        elif choice == 7:  # Change Location
            new_location = input("\nEnter a new location (city name): ")
            print(f"\nFetching weather data for {new_location}...")
            new_weather_data = get_weather_data(new_location)

            if "error" in new_weather_data:
                print(f"\nError: {new_weather_data['error']}")
                input("Press Enter to continue...")
            else:
                weather_data = new_weather_data
                print(f"\nLocation changed to {weather_data['location']['name']}, {weather_data['location']['country']}")
                input("Press Enter to continue...")

        elif choice == 8:  # Exit
            print("\nThank you for using the Weather Advisor Application. Goodbye!")
            break

            ## Note for Colab Usage
The `run_weather_advisor()` function above is designed for a terminal environment with continuous user interaction. In Google Colab, running this function directly may not work as expected due to input handling and display limitations. For demonstration purposes, individual components can be tested in the 'Testing and Examples' section below, or the function can be adapted for Colab with manual input cells.


In [ ]:
# Testing and Examples
This section demonstrates key features of the WeatherWise Advisor app with sample inputs and outputs. Each example tests a specific functionality, such as fetching weather data, generating visualizations, or processing natural language queries. Run these cells to see the app in action within the Colab environment.

In [ ]:
# Example 1: Fetch Weather Data for a Sample Location
location = "Sydney"
print(f"Fetching weather data for {location}...")
weather_data = get_weather_data(location)

if "error" in weather_data:
    print(f"Error: {weather_data['error']}")
else:
    print(f"Successfully retrieved data for {weather_data['location']['name']}, {weather_data['location']['country']}")
    print(f"Current Temperature: {weather_data['current']['temperature']}°C")
    print(f"Conditions: {weather_data['current']['description'].capitalize()}")


In [ ]:
# Example 2: Display Current Weather
if "error" not in weather_data:
    display_current_weather(weather_data)
else:
    print("Cannot display current weather due to earlier error.")


In [ ]:
# Example 3: Display Forecast
if "error" not in weather_data:
    display_forecast(weather_data)
else:
    print("Cannot display forecast due to earlier error.")


In [ ]:
# Example 4: Generate Temperature Visualization
if "error" not in weather_data:
    create_temperature_visualisation(weather_data)
else:
    print("Cannot generate visualization due to earlier error.")


In [ ]:
# Example 5: Generate Precipitation Visualization
if "error" not in weather_data:
    create_precipitation_visualisation(weather_data)
else:
    print("Cannot generate visualization due to earlier error.")


In [ ]:
# Example 6: Generate Wind Visualization
if "error" not in weather_data:
    create_wind_visualisation(weather_data)
else:
    print("Cannot generate visualization due to earlier error.")


In [ ]:
# Example 7: Process a Natural Language Weather Question
question = "Will it rain in Sydney tomorrow?"
print(f"Processing question: {question}")
parsed_question = parse_weather_question(question)

# If no location was found in the question, use the current one
if not parsed_question["location"]:
    parsed_question["location"] = weather_data["location"]["name"]

# Check if weather data is for the correct location
if (parsed_question["location"].lower() != weather_data["location"]["name"].lower() and
    parsed_question["location"].lower() != f"{weather_data['location']['name']}, {weather_data['location']['country']}".lower()):
    print(f"Fetching weather data for {parsed_question['location']}...")
    new_weather_data = get_weather_data(parsed_question["location"])
    if "error" not in new_weather_data:
        weather_data = new_weather_data
    else:
        print(f"Couldn't find weather data for {parsed_question['location']}. Using current location instead.")

response = generate_weather_response(parsed_question, weather_data)
print(f"\nResponse: {response}")
